# 1. Installation of Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 2. Import Transformed Bigdata

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
if os.getcwd() == '/content':
  os.chdir('drive/MyDrive/DataTeam1/Index2')

Mounted at /content/drive


In [57]:
df = pd.read_csv('current.csv')
df.drop([0],axis=0,inplace=True)

df["sasdate"] = pd.to_datetime(df["sasdate"])
df = df.set_index('sasdate') 

print(df.shape)
df.head()

(772, 127)


,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
1959-01-01,2442.158,2293.2,17.272,292266.4261,18235.77392,21.9665,23.3891,22.2688,31.7011,19.0149,...,17.791,11.326,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
1959-02-01,2451.778,2301.5,17.452,294424.7425,18369.56308,22.3966,23.7048,22.4617,31.9337,19.1147,...,17.798,11.343,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
1959-03-01,2467.594,2318.5,17.617,293418.6704,18523.05762,22.7193,23.8483,22.5719,31.9337,19.4890,...,17.785,11.363,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
1959-04-01,2483.671,2334.9,17.553,299322.8039,18534.46600,23.2032,24.1927,22.9026,32.4374,19.6138,...,17.796,11.403,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN
1959-05-01,2498.026,2350.4,17.765,301364.3249,18679.66354,23.5528,24.3936,23.1231,32.5925,20.0130,...,17.777,11.421,2.17,2.48,2.08,95.3,6753.0,12646.0,80.7315,NaN


Preliminarily, we will mainly consider 4 core indexes:
1. ISM Manufacturing: Production Index (NAPMPI) (will do, but now is missing)
2. Total non-farm payroll growth (PAYEMS)
3. 10-year treasury rate minus Fed funds rate (T10YFFM)
4. S&P's Common Stock Price Index: Composite (SP500)

Ref: https://s3.amazonaws.com/real.stlouisfed.org/wp/2015/2015-012.pdf

In [58]:
df = df[['PAYEMS','T10YFFM','S&P 500']]
df.isna().sum()
df

,PAYEMS,T10YFFM,S&P 500
sasdate,,,
1959-01-01,52478.0,1.54,55.62
1959-02-01,52688.0,1.53,54.77
1959-03-01,53014.0,1.19,56.16
1959-04-01,53321.0,1.16,57.10
1959-05-01,53550.0,1.41,57.96
...,...,...,...
2022-12-01,154535.0,-0.47,3912.38
2023-01-01,155007.0,-0.80,3960.66
2023-02-01,155333.0,-0.82,4079.68


In [59]:
df.dropna(inplace=True)
df

,PAYEMS,T10YFFM,S&P 500
sasdate,,,
1959-01-01,52478.0,1.54,55.62
1959-02-01,52688.0,1.53,54.77
1959-03-01,53014.0,1.19,56.16
1959-04-01,53321.0,1.16,57.10
1959-05-01,53550.0,1.41,57.96
...,...,...,...
2022-11-01,154296.0,0.14,3917.49
2022-12-01,154535.0,-0.47,3912.38
2023-01-01,155007.0,-0.80,3960.66


# Recession Labels

In [60]:
recession = pd.read_csv('USREC.csv')
recession.columns = ['DATE', 'USREC']
recession['DATE'] = pd.to_datetime(recession['DATE'])
recession = recession.set_index('DATE') 
recession

,USREC
DATE,
1959-01-01,0
1959-02-01,0
1959-03-01,0
1959-04-01,0
1959-05-01,0
...,...
2022-12-01,0
2023-01-01,0
2023-02-01,0


In [69]:
recession.drop(['2023-04-01'], axis=0, inplace=True)

# 3. Probabilistic models
1.  Gaussian Naive Bayes (G-NB)
2. Markov switching Naive Bayes (MS-NB)

In [73]:
X = df
y = recession

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=32)

### 3.1 G-NB

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Implemented model
y_pred = GaussianNB().fit(X_train, y_train).predict(X_test)
acc = np.sum(y_test['USREC']==y_pred)/len(y_test)
print(acc)

0.9096774193548387


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
# Model from scratch

class NaiveBayes:
  def fit(self, X, y):
    n_samples, n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)

    self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
    self._var = np.zeros((n_classes, n_features), dtype=np.float64)
    self._priors = np.zeros(n_classes, dtype=np.float64)

    for idx, c in enumerate(self._classes):
      X_c = X[y==c]
      self._mean[idx,:] = X_c.mean(axis=0)
      self._var[idx,:] = X_c.var(axis=0)
      self._priors[idx] = X_c.shape[0]/ float(n_samples)

  def predict(self,X):
    y_pred = [self._predict(x) for x in X]
    return np.array(y_pred)

  def _predict(self, x):
    posteriors = []

    for idx, c in enumerate(self._classes):
      prior = np.log(self._priors[idx])
      posterior = np.sum(np.log(self._pdf(idx,x)))
      posterior = posterior + prior
      posteriors.append(posterior)

    return self._classes[np.argmax(posteriors)]

  def _pdf(self, class_idx, x):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    numerator = np.exp(-((x-mean)**2)/(2*var))
    denominator = np.sqrt(2*np.pi*var)
    return numerator / denominator

In [82]:
nb = NaiveBayes()
nb.fit(X_train,y_train)
y_pred_scratch = nb.predict(X_test)
acc_scratch = np.sum(y_test['USREC']==y_pred_scratch)/len(y_test)
print(acc_scratch)

UFuncTypeError: ignored